# Bert Disaster Check

Source: https://www.kaggle.com/xhlulu/disaster-nlp-keras-bert-using-tfhub/notebook

# Setup

In [7]:
!pip install gradio
!pip install wandb
!pip install sentencepiece

     |████████████████████████████████| 1.2 MB 8.1 MB/s 


In [8]:
# Flexible integration for any Python script
import wandb
from wandb.keras import WandbCallback

# 1. Start a W&B run
wandb.init(project='disaster', entity='wasaequreshi')

# 2. Save model inputs and hyperparameters
config = wandb.config
config.learning_rate = 0.01   

In [9]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [10]:
!cp '/content/drive/My Drive/297 2021/hw_3/hw_3_bert_disaster/train.csv' .
!cp '/content/drive/My Drive/297 2021/hw_3/hw_3_bert_disaster/test.csv' .
!cp '/content/drive/My Drive/297 2021/hw_3/hw_3_bert_disaster/sample_submission.csv' .

# Bert Setup

In [11]:
!wget --quiet https://raw.githubusercontent.com/tensorflow/models/master/official/nlp/bert/tokenization.py


In [12]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint
import tensorflow_hub as hub

import tokenization

In [13]:
def bert_encode(texts, tokenizer, max_len=512):
    all_tokens = []
    all_masks = []
    all_segments = []
    
    for text in texts:
        text = tokenizer.tokenize(text)
            
        text = text[:max_len-2]
        input_sequence = ["[CLS]"] + text + ["[SEP]"]
        pad_len = max_len - len(input_sequence)
        
        tokens = tokenizer.convert_tokens_to_ids(input_sequence)
        tokens += [0] * pad_len
        pad_masks = [1] * len(input_sequence) + [0] * pad_len
        segment_ids = [0] * max_len
        
        all_tokens.append(tokens)
        all_masks.append(pad_masks)
        all_segments.append(segment_ids)
    
    return np.array(all_tokens), np.array(all_masks), np.array(all_segments)

In [14]:
def build_model(bert_layer, max_len=512):
    input_word_ids = Input(shape=(max_len,), dtype=tf.int32, name="input_word_ids")
    input_mask = Input(shape=(max_len,), dtype=tf.int32, name="input_mask")
    segment_ids = Input(shape=(max_len,), dtype=tf.int32, name="segment_ids")

    _, sequence_output = bert_layer([input_word_ids, input_mask, segment_ids])
    clf_output = sequence_output[:, 0, :]
    out = Dense(1, activation='sigmoid')(clf_output)
    
    model = Model(inputs=[input_word_ids, input_mask, segment_ids], outputs=out)
    model.compile(Adam(lr=1e-5), loss='binary_crossentropy', metrics=['accuracy'])
    
    return model

In [15]:
module_url = "https://tfhub.dev/tensorflow/bert_en_uncased_L-24_H-1024_A-16/1"
bert_layer = hub.KerasLayer(module_url, trainable=True)

# Data Load and Preprocessing

In [16]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
submission = pd.read_csv("sample_submission.csv")

In [17]:
vocab_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
do_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
tokenizer = tokenization.FullTokenizer(vocab_file, do_lower_case)

In [18]:
train_input = bert_encode(train.text.values, tokenizer, max_len=160)
test_input = bert_encode(test.text.values, tokenizer, max_len=160)
train_labels = train.target.values

# Build Bert Model

In [19]:
model = build_model(bert_layer, max_len=160)
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_word_ids (InputLayer)     [(None, 160)]        0                                            
__________________________________________________________________________________________________
input_mask (InputLayer)         [(None, 160)]        0                                            
__________________________________________________________________________________________________
segment_ids (InputLayer)        [(None, 160)]        0                                            
__________________________________________________________________________________________________
keras_layer (KerasLayer)        [(None, 1024), (None 335141889   input_word_ids[0][0]             
                                                                 input_mask[0][0]             

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


In [ ]:
checkpoint = ModelCheckpoint('model.h5', monitor='val_loss', save_best_only=True)

train_history = model.fit(
    train_input, train_labels,
    validation_split=0.2,
    epochs=1,
    callbacks=[checkpoint, WandbCallback(monitor="val_loss", verbose=0, mode="auto", save_weights_only=(False),    log_weights=(False), log_gradients=(False), save_model=(True),    training_data=None, validation_data=None, labels=[], data_type=None,    predictions=36, generator=None, input_type=None, output_type=None,    log_evaluation=(False), validation_steps=None, class_colors=None,    log_batch_frequency=None, log_best_prefix="best_", save_graph=(True),    validation_indexes=None, validation_row_processor=None,    prediction_row_processor=None, infer_missing_processors=(True),    log_evaluation_frequency=0)],
    batch_size=16
)

371/381 [============================>.] - ETA: 9s - loss: 0.4330 - accuracy: 0.8135

# Interactive

In [29]:
import gradio as gr

def greet(input_sentence):
  res = bert_encode([input_sentence], tokenizer, max_len=160)
  result = round(model.predict(res)[0][0])
  if result == 0:
    return "Not Disaster"
  else:
    return "Disaster"

iface = gr.Interface(fn=greet, inputs="text", outputs="text")
iface.launch()

Colab notebook detected. To show errors in colab notebook, set `debug=True` in `launch()`
This share link will expire in 72 hours. If you need a permanent link, visit: https://gradio.app/introducing-hosted
Running on External URL: https://19091.gradio.app


(<Flask 'gradio.networking'>,
 'http://127.0.0.1:7861/',
 'https://19091.gradio.app')